Import packages 

In [1]:
#Importing packages
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

Define the Hyperparameter & Data info

In [2]:
#Learning Rate, Iterations, Batch Size Hyperparameters
learning_rate = 0.001
iterations = 20000
batch_size = 256
#dropout = 0.5
#epochs = 10

Load CIFAR-10 Data

In [3]:
#CIFAR-10 Dataset has 60000 images of common objects, 6k images per class and 10 classes in total
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

Data Preparation

In [4]:
#Convert to float type
X_train, X_test = np.array(X_train, np.float32), np.array(X_test, np.float32)

#Flatten images to 1-D vector of 3072 features (32*32*3)
X_train, X_test = X_train.reshape([-1, 3072]), X_test.reshape([-1, 3072])

#One hot encoding of labels
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

#Normalization of images
X_train = X_train / 255.
X_test = X_test / 255.

#Use Tensorflow data for shuffling and fetching it batchwise
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(50000, 3072)
(50000, 10)
(10000, 3072)
(10000, 10)


Define the model

In [5]:
class NeuralNet(Model):
    
    def __init__(self, use_dp = True):
        super(NeuralNet, self).__init__()
        self.use_dp = use_dp        
        self.L1 = layers.Dense(3072, activation = tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))
        self.L2 = layers.Dense(2048, activation = tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))
        self.L3 = layers.Dense(1024, activation = tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))
        self.L4 = layers.Dense(512, activation = tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))
        self.L5 = layers.Dense(256, activation = tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))
        self.L6 = layers.Dense(128, activation = tf.nn.relu, kernel_regularizer = tf.keras.regularizers.l2(0.01))
        if self.use_dp:
            self.dp = layers.Dropout(0.5)
        self.Yhat = layers.Dense(10)

    # Set forward pass.
    def call(self, x, is_training=False, use_dp = True):
        x = self.L1(x)
        if self.use_dp:
            x = self.dp(x)
        x = self.L2(x)
        if self.use_dp:
            x = self.dp(x)
        x = self.L3(x)
        if self.use_dp:
            x = self.dp(x)
        x = self.L4(x)
        if self.use_dp:
            x = self.dp(x)
        x = self.L5(x)
        if self.use_dp:
            x = self.dp(x)
        x = self.L6(x)
        if self.use_dp:
            x = self.dp(x)
        x = self.Yhat(x)
        if not is_training:
            x = tf.nn.softmax(x)
        return x

# Build the network
model = NeuralNet()

Define the cost function

In [6]:
#Cost function definition, using the softmax cross entropy loss
def cross_entropy(y_pred, y_true):
    
    loss = tf.nn.softmax_cross_entropy_with_logits(logits = y_pred, labels = y_true)
    
    return tf.reduce_mean(loss)

Metrics to check predictions

In [7]:
#To verify the predictions
def accuracy(y_pred, y_true):
    
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
    a = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)
    
    return a

Define the optimizer

In [8]:
#Setup the optimizer function with learning rate as parameter
optimizer = tf.optimizers.Adam(learning_rate)

Put it all together (Optimization of the network)

In [9]:
def run(X, Y):
    
    #Using GradientTape for automatic differentiation.
    with tf.GradientTape() as tape:
        pred = model(X, is_training = True, use_dp = True)
        loss = cross_entropy(pred, Y)
        
    #Get W and B values 
    train_variables = model.trainable_variables
    
    #Compute gradients.
    gradients = tape.gradient(loss, train_variables)
    
    #Update W and b following gradients
    optimizer.apply_gradients(zip(gradients, train_variables))

Let's train for 40000 iterations

In [10]:
#Run the for-loop for a well defined number of epochs
s = time.clock()
loss_plot = []
accuracy_plot = []

for i, (Batch_x, Batch_y) in enumerate(train_data.take(20001), 1):
    #Run the optimization to update W and b values.
    run(Batch_x, Batch_y)
    
    if i % 1000 == 0:
        pred = model(Batch_x, is_training = True, use_dp = False)
        loss = cross_entropy(pred, Batch_y)
        acc = accuracy(pred, Batch_y)
        print('Iteration = {}'.format(i))
        print('Loss = {}'.format(loss))
        print('Accuracy = {}\n'.format(acc))
        
        loss_plot.append(loss)
        accuracy_plot.append(acc)
        
e = time.clock()

print('Time Elapsed = {} \n'.format(e-s))

pred = model(X_test, is_training = False, use_dp = False)
test_accuracy = accuracy(pred, Y_test)
print('Accuracy on Test Set is {}'.format(test_accuracy))

C:\Users\Pranav\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


Iteration = 1000
Loss = 1.3725861310958862
Accuracy = 0.5078125

Iteration = 2000
Loss = 1.3023344278335571
Accuracy = 0.515625

Iteration = 3000
Loss = 0.9607739448547363
Accuracy = 0.64453125

Iteration = 4000
Loss = 0.7955528497695923
Accuracy = 0.73046875

Iteration = 5000
Loss = 0.5276813507080078
Accuracy = 0.8203125

Iteration = 6000
Loss = 0.3398495316505432
Accuracy = 0.87890625

Iteration = 7000
Loss = 0.19062215089797974
Accuracy = 0.9609375

Iteration = 8000
Loss = 0.07492822408676147
Accuracy = 0.9765625

Iteration = 9000
Loss = 0.09678030759096146
Accuracy = 0.98046875

Iteration = 10000
Loss = 0.04686673730611801
Accuracy = 0.984375

Iteration = 11000
Loss = 0.054417334496974945
Accuracy = 0.98046875



KeyboardInterrupt: 

Plotting the graph

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

iterations = np.arange(1000, 20001, 1000)
plt.plot(iterations, loss_plot, label = 'Training Loss')
plt.title('Loss graph')
plt.xlabel('Epochs')
plt.xticks(np.arange(1000, 20001, 1000))
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
iterations = np.arange(1000, 20001, 1000)
plt.plot(iterations, accuracy_plot, label = 'Training Accuracy')
plt.title('Accuracy graph')
plt.xlabel('Epochs')
plt.xticks(np.arange(1000, 20001, 1000))
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Real Time Predictions

In [ ]:
def real_time(index):
    '''
    
    '''
    
    labels = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
    
    prediction = model(X_test[index].reshape(1, 3072), is_training = False, use_dp = False)
    print('Actual Ground Truth is {} ; {}'.format(np.squeeze(Y_test[index]), labels[np.argmax(Y_test[index])]))
    print('Prediction for test image is {}; {}'.format(np.squeeze(prediction), labels[np.argmax(prediction[0])]))
    plt.imshow(np.reshape(X_test[index].reshape(1, 3072), [32, 32, 3]))

In [ ]:
real_time(20)

In [ ]:
real_time(40)

In [ ]:
real_time(100)